# Maryam's web scraping assignment

## WEB SCRAPING

In [ ]:
import sys
!{sys.executable} -m pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
print('Libraries Imported!')

In [ ]:
source= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())
tr= soup.find('tr')
print(tr.prettify())


In [ ]:
source= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

csv_file= open('postal_code_canada.csv', 'w')
csv_writer= csv.writer(csv_file)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighborhood'])

for tr in soup.find_all('tr'):
    try:
        postal_code= tr.td.text
    except Exception as e:
        postal_code= None
    print(postal_code)
    
    try:
        borough= tr.a['title']
    except Exception as e:
        borough= None
    print(borough)
    
    try:
        neighborhood= tr.select('a')[1]['title']
        
    except Exception as e:
        neighborhood= None
    print(neighborhood)
    
    print()

    
    csv_writer.writerow([postal_code, borough, neighborhood])
    
csv_file.close()

### READ DATA AND PREPROCESS

In [12]:
import pandas as pd
import numpy as np

In [16]:
canada_df= pd.read_csv('postal_code_canada.csv')
canada_df.head()

,PostalCode,Borough,Neighborhood
0,NaN,NaN,NaN
1,M1A,NaN,NaN
2,M2A,NaN,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [18]:
canada_df = canada_df.dropna(subset=['Borough'])
canada_df.tail()

,PostalCode,Borough,Neighborhood
287,M8Z,Etobicoke,NaN
290,\n,Flag of Canada,Postal codes in Canada
291,\n\n\nNL\n\nNS\n\nPE\n\nNB\n\nQC\n\nON\n\nMB\n...,Newfoundland and Labrador,Nova Scotia
292,NL\n,Newfoundland and Labrador,Nova Scotia
293,A\n,List of postal codes of Canada: A,List of postal codes of Canada: B


Delete the last 4 rows which are non essential

In [19]:
canada_df= canada_df.drop([290, 291, 292, 293], axis=0)
canada_df.tail()

,PostalCode,Borough,Neighborhood
283,M8Z,Etobicoke,NaN
284,M8Z,Etobicoke,Mimico
285,M8Z,Etobicoke,The Queensway
286,M8Z,Etobicoke,NaN
287,M8Z,Etobicoke,NaN


*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough*. So i replaced all the NaN in **Neighborhood** with the **Borough** names.

In [20]:
canada_df= canada_df.mask(canada_df.eq(-1)).ffill(axis=1)
canada_df.tail()

,PostalCode,Borough,Neighborhood
283,M8Z,Etobicoke,Etobicoke
284,M8Z,Etobicoke,Mimico
285,M8Z,Etobicoke,The Queensway
286,M8Z,Etobicoke,Etobicoke
287,M8Z,Etobicoke,Etobicoke


There are duplicates in the DataFrame. i will remove them with the assumption that Neighborhood names are uinique.

In [22]:
canada_df = canada_df.drop_duplicates(subset=['Neighborhood'])
canada_df.tail()

,PostalCode,Borough,Neighborhood
270,M8Y,Etobicoke,Mimico
271,M8Y,Etobicoke,"Old Mill, Toronto"
272,M8Y,Etobicoke,The Queensway
273,M8Y,Etobicoke,Fairmont Royal York Hotel
274,M8Y,Etobicoke,Sunnylea


In [23]:
canada_df.shape

(133, 3)